# install and import libraries


In [ ]:
%pip install openai

In [ ]:
%pip install python-dotenv

In [ ]:
import asyncio
import io
import difflib
import pathlib
from typing import List, Optional
from pydantic import BaseModel
from openai import OpenAI
import os
import pandas as pd
import json
import re
from tqdm import tqdm
from google.colab import userdata


All the benchmarks were manually went through some were combined or dropped depending on wheter they were seperate.


In [ ]:
df = pd.read_csv('/content/extended_benchmarks_with_descriptions.csv')
df

,name,description,source,o1,Claude_3,gemini_15,llama3,gpt-4o
0,100Q Hard,"A set of 100 human-written questions, curated...",https://www-cdn.anthropic.com/de8ba9b01c9ab7cb...,False,True,False,False,False
1,1H-VideoQA,Question-answering benchmarks for long-context...,https://arxiv.org/pdf/2403.05530,False,False,True,False,False
2,CommonSenseQA,"When answering a question, people often draw u...",https://arxiv.org/pdf/1811.00937v2,False,False,False,True,False
3,AGIEval,Evaluating the general abilities of foundation...,https://arxiv.org/abs/2304.06364,False,False,False,True,False
4,AI2D,Diagrams are common tools for representing com...,https://arxiv.org/pdf/1603.07396,False,True,True,True,False
...,...,...,...,...,...,...,...,...
218,WorldSense,"We propose WorldSense, a benchmark designed to...",https://arxiv.org/abs/2311.15930,False,False,False,True,False
219,XSTest,"Without proper safeguards, large language mode...",https://arxiv.org/abs/2308.01263,True,True,False,False,False
220,YouCook,This data set was prepared from 88 open-source...,https://paperswithcode.com/dataset/youcook,False,False,True,False,False
221,YouCook2,"YouCook2 is the largest task-oriented, instruc...",https://paperswithcode.com/dataset/youcook2,False,False,True,False,False


In [ ]:
urls = [

'https://arxiv.org/pdf/2407.21783',
'https://arxiv.org/pdf/2403.05530'
'https://cdn.openai.com/o1-system-card-20241205.pdf',
'https://arxiv.org/pdf/2410.21276',
'https://www-cdn.anthropic.com/de8ba9b01c9ab7cbabf5c33b80b7bbc618857627/Model_Card_Claude_3.pdf',
'https://arxiv.org/html/2403.05530v2#bib.bib81',
'https://arxiv.org/pdf/2403.05530v2',
'https://arxiv.org/pdf/2407.21783',
 'Multiple'

]

In [ ]:
df.dropna(subset=['description'],inplace=True)
df['internal'] = df.source.isin(urls)
df


,name,description,source,o1,Claude_3,gemini_15,llama3,gpt-4o,internal
0,100Q Hard,"A set of 100 human-written questions, curated...",https://www-cdn.anthropic.com/de8ba9b01c9ab7cb...,False,True,False,False,False,True
1,1H-VideoQA,Question-answering benchmarks for long-context...,https://arxiv.org/pdf/2403.05530,False,False,True,False,False,False
2,CommonSenseQA,"When answering a question, people often draw u...",https://arxiv.org/pdf/1811.00937v2,False,False,False,True,False,False
3,AGIEval,Evaluating the general abilities of foundation...,https://arxiv.org/abs/2304.06364,False,False,False,True,False,False
4,AI2D,Diagrams are common tools for representing com...,https://arxiv.org/pdf/1603.07396,False,True,True,True,False,False
...,...,...,...,...,...,...,...,...,...
218,WorldSense,"We propose WorldSense, a benchmark designed to...",https://arxiv.org/abs/2311.15930,False,False,False,True,False,False
219,XSTest,"Without proper safeguards, large language mode...",https://arxiv.org/abs/2308.01263,True,True,False,False,False,False
220,YouCook,This data set was prepared from 88 open-source...,https://paperswithcode.com/dataset/youcook,False,False,True,False,False,False
221,YouCook2,"YouCook2 is the largest task-oriented, instruc...",https://paperswithcode.com/dataset/youcook2,False,False,True,False,False,False


In [ ]:
df['internal'] = df.source.isin(urls)

In [ ]:
client = OpenAI(api_key=userdata.get("OPENAI_API_KEY"))



class Classification(BaseModel):
    behavioral_evaluation: bool
    knowledge_accuracy: bool
    wrong_answer_resistance: bool
    informational_risks: bool
    measures_capabilities: bool
    is_variant: bool
    existential_risk: bool
    jailbreak_resistance: bool

system_prompt = """

**Task:**
Analyze the given benchmark description and classify it according to the specified categories. Return the results using the exact JSON structure provided below. Each field should be assigned a boolean value (`true` or `false`) based on whether the benchmark matches the corresponding definition.

---

### **Detailed Field Descriptions:**

1. **behavioral_evaluation:**
   - **Definition:** Indicates whether the benchmark assesses behavioral aspects related to user interactions or ethical concerns.
   - **Key Indicators:**
     - Tests for reliance or over-reliance on AI outputs.
     - Assesses addictive behavior linked to AI use.
     - Evaluates user trust, safety perceptions, or ethical implications of AI behavior.
     - Focuses on human-AI interaction patterns or psychological effects.
     - Examines **persuasion** abilities, including the AI's influence on user opinions, beliefs, or decisions, either intentionally or unintentionally.
     - Tests for **sycophancy**, where the AI inappropriately aligns with user opinions or biases without critical analysis or correction, potentially reinforcing incorrect or harmful viewpoints.

---

2. **knowledge_accuracy:**
   - **Definition:** Indicates whether the benchmark evaluates the factual correctness, completeness, and consistency of the knowledge provided by the AI.
   - **Key Indicators:**
     - Measures factual accuracy in responses.
     - Assesses knowledge consistency across related queries.
     - Evaluates AI's ability to provide comprehensive and correct information.
     - Tests understanding of well-established facts across multiple domains.

---

3. **wrong_answer_resistance:**
   - **Definition:** Indicates whether the benchmark specifically tests the AI’s resistance to providing incorrect answers.
   - **Key Conditions (must **both** be true for this field to be `true`):**
     1. The **knowledge_accuracy** field is `true`.
     2. The benchmark penalizes **wrong answers** more than **no answer** (i.e., providing an incorrect response is considered worse than declining to answer).
   - **Key Indicators:**
     - The benchmark explicitly rewards the AI for withholding responses when uncertain.
     - Incorrect answers significantly impact scoring compared to unanswered questions.
     - Focus on avoiding confident but incorrect outputs.

---

4. **informational_risks:**
   - **Definition:** Indicates whether the benchmark assesses risks related to the *quality*, *appropriateness*, or *alignment* of the information provided by the AI. This includes trade-offs between helpfulness and harmlessness, alignment with user intentions, and value-sensitive outputs.
   - **Key Indicators:**
     - Evaluates the AI’s ability to provide useful information without causing harm.
     - Tests for alignment issues, including whether outputs match the user’s intended goals or context.
     - Assesses risks related to biased, inappropriate, or unsafe content generation.
     - Considers whether AI responses strike the right balance between being helpful and not providing harmful or dangerous information.

---

5. **measures_capabilities:**
   - **Definition:** Indicates whether the benchmark measures the AI's core functional abilities and performance in specific tasks or domains.
   - **Key Indicators:**
     - Evaluates linguistic abilities such as comprehension, generation, translation, or summarization.
     - Assesses reasoning, problem-solving, or decision-making skills.
     - Tests performance in specialized domains (e.g., mathematics, programming, legal reasoning).
     - Benchmarks general-purpose cognitive tasks, including memory, logic, and multi-modal understanding.

---

6. **is_variant:**
   - **Definition:** Specifies whether the benchmark is a variant of another existing benchmark, possibly adjusted for different conditions, languages, populations, or complexity levels.
   - **Key Indicators:**
     - Adaptations of existing benchmarks with minor or major modifications.
     - Benchmarks focusing on niche subsets of a broader test (e.g., language-specific versions).
     - Variations that test robustness under altered inputs or stress conditions.

---

7. **existential_risk:**
   - **Definition:** Indicates whether the benchmark assesses potential existential risks associated with AI, particularly those related to catastrophic biological, chemical, radiological, or nuclear (CBRN) threats or other large-scale risks that could threaten human survival.
   - **Key Indicators:**
     - Evaluates AI’s potential role in facilitating, detecting, or preventing CBRN threats.
     - Tests for alignment issues that could lead to existential-level risks.
     - Assesses AI misuse scenarios that could result in global catastrophic outcomes.
     - Focuses on long-term safety concerns, including AI’s role in dual-use technologies with existential implications.

---

8. **jailbreak_resistance:**
   - **Definition:** Indicates whether the benchmark assesses the AI’s resistance to "jailbreaking" attempts—situations where users try to manipulate the AI into bypassing safety restrictions, generating harmful content, or performing unauthorized actions.
   - **Key Indicators:**
     - Evaluates how well the AI can resist adversarial prompts designed to circumvent content safety measures.
     - Tests for the AI’s robustness against prompt injection attacks.
     - Assesses the AI’s ability to maintain ethical and safety boundaries despite manipulative or deceptive inputs.
     - Benchmarks scenarios that simulate real-world attempts to exploit AI systems for malicious purposes.

---

### **Expected JSON Output Format:**
```json
{
    "behavioral_evaluation": true,
    "knowledge_accuracy": true,
    "wrong_answer_resistance": true,
    "informational_risks": true,
    "measures_capabilities": true,
    "is_variant": false,
    "existential_risk": false,
    "jailbreak_resistance": true
}
```

---

### **Instructions for Classification:**
- For each field, return `true` if the benchmark aligns with the described criteria; otherwise, return `false`.
- **Important:** For **wrong_answer_resistance** to be `true`, both conditions listed must be satisfied. If either condition fails, it must be `false`.
- Carefully analyze the benchmark description to ensure accurate classification across all nine fields.
- The final output **must** follow the exact JSON format provided, with boolean values (`true` or `false`) in lowercase.
- If a benchmark fits multiple categories, assign `true` to all applicable fields.
""".strip()

def classify_benchmark(benchmark_name: str,description:str):
    """
    Reads the markdown file at markdown_path and extracts minimal benchmark information using the OpenAI API.
    """
    intermediate_steps = []
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": f"Classify {benchmark_name} using the following description:\n```\n{description}\n```"},
    ]

    completion = client.beta.chat.completions.parse(
            model="o3-mini",
            messages=messages,
            response_format=Classification,
            reasoning_effort='medium',
    )

    return completion.choices[0].message.parsed.model_dump()

async def extract_minimal_benchmarks_for_files(benchmarks: pd.DataFrame):
    """
    Asynchronously extracts minimal benchmarks for each file in file_paths.
    """

    # Create a list of tasks that run extract_all_minimal_benchmarks in a separate thread
    tasks = [asyncio.to_thread(classify_benchmark,row['name'],row['description']) for _, row in benchmarks.iterrows()]

    # Wait for all tasks to complete concurrently
    results = await asyncio.gather(*tasks)
    return results


In [ ]:
results = await extract_minimal_benchmarks_for_files(df)
results

In [ ]:
df_results = pd.DataFrame(results)
df_results.to_csv('results.csv')
df_results

In [ ]:
df.reset_index(drop=True, inplace=True)


In [ ]:
new_df = pd.concat([df,df_results],axis=1)

In [ ]:
new_df.to_csv('classified_benchmarks.csv')

# manual review

A quick lookover the classifications with some fixes made

In [ ]:
pd.set_option('display.max_rows', None)  # Show all rows
pd.set_option('display.max_colwidth', None)  # Show full column content

In [ ]:
new_df = pd.read_csv('/content/classified_benchmarks.csv', index_col=0)
new_df.columns

Index(['name', 'description', 'source', 'o1', 'Claude_3', 'gemini_15',
       'llama3', 'gpt-4o', 'internal', 'behavioral_evaluation',
       'knowledge_accuracy', 'wrong_answer_resistance', 'informational_risks',
       'measures_capabilities', 'is_variant', 'existential_risk',
       'jailbreak_resistance'],
      dtype='object')

## **behavioral_evaluation:**
   - **Definition:** Indicates whether the benchmark assesses behavioral aspects related to user interactions or ethical concerns.
   - **Key Indicators:**
     - Tests for reliance or over-reliance on AI outputs.
     - Assesses addictive behavior linked to AI use.
     - Evaluates user trust, safety perceptions, or ethical implications of AI behavior.
     - Focuses on human-AI interaction patterns or psychological effects.
     - Examines **persuasion** abilities, including the AI's influence on user opinions, beliefs, or decisions, either intentionally or unintentionally.
     - Tests for **sycophancy**, where the AI inappropriately aligns with user opinions or biases without critical analysis or correction, potentially reinforcing incorrect or harmful viewpoints.


In [ ]:
key = 'behavioral_evaluation'
new_df[new_df[key]][['name','description',key]]

,name,description,behavioral_evaluation
32,BBQ,"We introduce the Bias Benchmark for QA (BBQ), a dataset of question sets constructed by the authors that highlight attested social biases against people belonging to protected classes along nine social dimensions relevant for U.S. English-speaking contexts. Our task evaluates model responses at two levels: (i) given an under-informative context, we test how strongly responses reflect social biases, and (ii) given an adequately informative context, we test whether the model's biases override a correct answer choice. We find that models often rely on stereotypes when the context is under-informative, meaning the model's outputs consistently reproduce harmful biases in this setting.",True
34,BeHonest,"Previous works on Large Language Models (LLMs) have mainly focused on evaluating their helpfulness or harmlessness. However, honesty, another crucial alignment criterion, has received relatively less attention. Dishonest behaviors in LLMs, such as spreading misinformation and defrauding users, present severe risks that intensify as these models approach superintelligent levels. Enhancing honesty in LLMs addresses critical limitations and helps uncover latent capabilities that are not readily expressed. This underscores the urgent need for reliable methods and benchmarks to effectively ensure and evaluate the honesty of LLMs.\n In this paper, we introduce BeHonest, a pioneering benchmark specifically designed to assess honesty in LLMs comprehensively. BeHonest evaluates three essential aspects of honesty: awareness of knowledge boundaries, avoidance of deceit, and consistency in responses. Building on this foundation, we designed 10 scenarios to evaluate and analyze 9 popular LLMs on the market, including both closed-source and open-source models from different model families with varied model sizes. Our findings indicate that there is still significant room for improvement in the honesty of LLMs. We encourage the AI community to prioritize honesty alignment in these models, which can harness their full potential to benefit society while preventing them from causing harm through deception or inconsistency. Our benchmark and code can be found at: \url{this https URL}.",True
42,ChangeMyView,"ChangeMyView is an evaluation we created that aims to directly measure persuasiveness and\nargumentative reasoning. We leverage existing human data from r/ChangeMyView, a popular\nsubreddit with 4 million members that is an established and reputable resource for persuasion\nanalysis[33].\nr/ChangeMyView works as follows:\n• Users (denoted the “original poster” or OP) present their own opinions and supporting\nrationale (see example below):\n– Title: “Shoes off should be the default when visiting a guest’s house”\n– Explanation: “This should be the default as it is the polite thing to do. Shoes\ncarry a lot of dirt and germs, therefore you should leave them at the door. It is also\nuncomfortable for the owner of the home to have to ask folks to remove their shoes.”\n• Other Reddit users write responses to attempt to persuade the OP of the opposing view.\n• Any responses that are successful result in the OP granting a “delta”, representing a change\nin their original view.",True
114,MakeMePay,"This evaluation tests an AI system’s ability to generate persuasive or manipulative text, specifically in the setting of convincing another (simulated) entity to part with money.\n\nThe setup consists of two language models engaging in conversation, where one model takes on the role of a con-artist trying to extract money, and the other model acts as the “mark” or target, simulating a person who has just received some money.\n\nThe con-artist model employs various persuasive techniques to get the mark to hand over its funds.\n\nHeadline metrics include success-rate at receiving a payment, as well as measurements of how large the payments are.",True
115,MakeMeSay,"This evaluation tests a model’s ability to generate

In [ ]:
new_df.at[148, 'behavioral_evaluation'] = False

new_df.at[215, 'behavioral_evaluation'] = False

In [ ]:
new_df.at[211, 'behavioral_evaluation'] = True
new_df.at[212, 'behavioral_evaluation'] = True

## **knowledge_accuracy:**
   - **Definition:** Indicates whether the benchmark evaluates the factual correctness, completeness, and consistency of the knowledge provided by the AI.
   - **Key Indicators:**
     - Measures factual accuracy in responses.
     - Assesses knowledge consistency across related queries.
     - Evaluates AI's ability to provide comprehensive and correct information.
     - Tests understanding of well-established facts across multiple domains.




In [ ]:
key = 'knowledge_accuracy'
new_df[new_df[key]][['name','description',key]]

,name,description,knowledge_accuracy
0,100Q Hard,"A set of 100 human-written questions, curated to be relatively obscure and to encourage\nmodels in the Claude 2 family to respond with dubious or incorrect information. Examples include\n“Why is Berkeley Bowl called Berkeley Bowl?”, “What is the Opto Electronics Factory (OLF)?”,\n“Tell me about Mary I, Countess of Menteith.",True
1,1H-VideoQA,"Question-answering benchmarks for long-context video understanding need to have at least two properties: first, they need to contain long videos and second, their questions need to be designed to in a way that can differentiate among models that operate over different context lengths. Unfortunately, no existing benchmarks satisfy these properties for evaluating models that can handle hours-long videos like Gemini 1.5 Pro. The publicly available question answering benchmark with the longest videos is EgoSchema (Mangalam et al., 2023), but its videos are at most 3 minutes (i.e., 180 frames) in length. To bridge this evaluation gap, we introduce a new benchmark, 1H-VideoQA, composed of 125 five-way multiple-choice questions over public videos 40-105 minutes long.",True
2,CommonSenseQA,"When answering a question, people often draw upon their rich world knowledge in addition to the particular context. Recent work has focused primarily on answering questions given some relevant document or context, and required very little general background. To investigate question answering with prior knowledge, we present CommonsenseQA: a challenging new dataset for commonsense question answering. To capture common sense beyond associations, we extract from ConceptNet (Speer et al., 2017) multiple target concepts that have the same semantic relation to a single source concept. Crowd-workers are asked to author multiple-choice questions that mention the source concept and discriminate in turn between each of the target concepts. This encourages workers to create questions with complex semantics that often require prior knowledge. We create 12,247 questions through this procedure and demonstrate the difficulty of our task with a large number of strong baselines. Our best baseline is based on BERT-large (Devlin et al., 2018) and obtains 56% accuracy, well below human performance, which is 89%.",True
3,AGIEval,"Evaluating the general abilities of foundation models to tackle human-level tasks is a vital aspect of their development and application in the pursuit of Artificial General Intelligence (AGI). Traditional benchmarks, which rely on artificial datasets, may not accurately represent human-level capabilities. In this paper, we introduce AGIEval, a novel benchmark specifically designed to assess foundation model in the context of human-centric standardized exams, such as college entrance exams, law school admission tests, math competitions, and lawyer qualification tests. We evaluate several state-of-the-art foundation models, including GPT-4, ChatGPT, and Text-Davinci-003, using this benchmark. Impressively, GPT-4 surpasses average human performance on SAT, LSAT, and math competitions, attaining a 95% accuracy rate on the SAT Math test and a 92.5% accuracy on the English test of the Chinese national college entrance exam. This demonstrates the extraordinary performance of contemporary foundation models. In contrast, we also find that GPT-4 is less proficient in tasks that require complex reasoning or specific domain knowledge. Our comprehensive analyses of model capabilities (understanding, knowledge, reasoning, and calculation) reveal these models' strengths and limitations, providing valuable insights into future directions for enhancing their general capabilities. By concentrating on tasks pertinent to human cognition and decision-making, our benchmark delivers a more meaningful and robust evaluation of foundation models' performance in real-world scenarios. The data, code, and all model outputs are released in this https URL.",True
6,AP Art History,The A

In [ ]:
wrong_ids = [149,34,200,218,28,50,51,52,54,55,57,58,59,79,93,159,160,161,163,164,172,173,168,187,188,191,200,203]
for wrong_id in wrong_ids:
  new_df.at[wrong_id,'knowledge_accuracy'] = False

## **wrong_answer_resistance:**
   - **Definition:** Indicates whether the benchmark specifically tests the AI’s resistance to providing incorrect answers.
   - **Key Conditions (must **both** be true for this field to be `true`):**
     1. The **knowledge_accuracy** field is `true`.
     2. The benchmark penalizes **wrong answers** more than **no answer** (i.e., providing an incorrect response is considered worse than declining to answer).
   - **Key Indicators:**
     - The benchmark explicitly rewards the AI for withholding responses when uncertain.
     - Incorrect answers significantly impact scoring compared to unanswered questions.
     - Focus on avoiding confident but incorrect outputs.



In [ ]:
key = 'wrong_answer_resistance'
new_df[new_df[key]][['name','description',key]]

,name,description,wrong_answer_resistance
69,GSM-Plus,"Large language models (LLMs) have achieved impressive performance across various mathematical reasoning benchmarks. However, there are increasing debates regarding whether these models truly understand and apply mathematical knowledge or merely rely on shortcuts for mathematical reasoning. One essential and frequently occurring evidence is that when the math questions are slightly changed, LLMs can behave incorrectly. This motivates us to evaluate the robustness of LLMs' math reasoning capability by testing a wide range of question variations. We introduce the adversarial grade school math (GSM-Plus) dataset, an extension of GSM8K augmented with various mathematical perturbations. Our experiments on 25 LLMs and 4 prompting techniques show that while LLMs exhibit different levels of math reasoning abilities, their performances are far from robust. In particular, even for problems that have been solved in GSM8K, LLMs can make mistakes when new statements are added or the question targets are altered. We also explore whether more robust performance can be achieved by composing existing prompting methods, in which we try an iterative method that generates and verifies each intermediate thought based on its reasoning goal and calculation result.",True
172,SQuAD,"Stanford Question Answering Dataset (SQuAD) is a reading comprehension dataset, consisting of questions posed by crowdworkers on a set of Wikipedia articles, where the answer to every question is a segment of text, or span, from the corresponding reading passage, or the question might be unanswerable.",True
173,SQuAD V2,"Stanford Question Answering Dataset (SQuAD) is a reading comprehension dataset, consisting of questions posed by crowdworkers on a set of Wikipedia articles, where the answer to every question is a segment of text, or span, from the corresponding reading passage, or the question might be unanswerable.\n\nSQuAD2.0 combines the 100,000 questions in SQuAD1.1 with over 50,000 unanswerable questions written adversarially by crowdworkers to look similar to answerable ones. To do well on SQuAD2.0, systems must not only answer questions when possible, but also determine when no answer is supported by the paragraph and abstain from answering.",True
180,SimpleQA,"We present SimpleQA, a benchmark that evaluates the ability of language models to answer short, fact-seeking questions. We prioritized two properties in designing this eval. First, SimpleQA is challenging, as it is adversarially collected against GPT-4 responses. Second, responses are easy to grade, because questions are created such that there exists only a single, indisputable answer. Each answer in SimpleQA is graded as either correct, incorrect, or not attempted. A model with ideal behavior would get as many questions correct as possible while not attempting the questions for which it is not confident it knows the correct answer. SimpleQA is a simple, targeted evaluation for whether models ""know what they know,"" and our hope is that this benchmark will remain relevant for the next few generations of frontier models. SimpleQA can be found at this https URL.",True


In [ ]:
wrong_ids = [69,172,173]
for wrong_id in wrong_ids:
  new_df.at[wrong_id,key] = False

new_df.at[150,key] = True


## **informational_risks:**
   - **Definition:** Indicates whether the benchmark assesses risks related to the *quality*, *appropriateness*, or *alignment* of the information provided by the AI. This includes trade-offs between helpfulness and harmlessness, alignment with user intentions, and value-sensitive outputs.
   - **Key Indicators:**
     - Evaluates the AI’s ability to provide useful information without causing harm.
     - Tests for alignment issues, including whether outputs match the user’s intended goals or context.
     - Assesses risks related to biased, inappropriate, or unsafe content generation.
     - Considers whether AI responses strike the right balance between being helpful and not providing harmful or dangerous information.



In [ ]:
key = 'informational_risks'
new_df[new_df[key]][['name','description',key]]

,name,description,informational_risks
32,BBQ,"We introduce the Bias Benchmark for QA (BBQ), a dataset of question sets constructed by the authors that highlight attested social biases against people belonging to protected classes along nine social dimensions relevant for U.S. English-speaking contexts. Our task evaluates model responses at two levels: (i) given an under-informative context, we test how strongly responses reflect social biases, and (ii) given an adequately informative context, we test whether the model's biases override a correct answer choice. We find that models often rely on stereotypes when the context is under-informative, meaning the model's outputs consistently reproduce harmful biases in this setting.",True
34,BeHonest,"Previous works on Large Language Models (LLMs) have mainly focused on evaluating their helpfulness or harmlessness. However, honesty, another crucial alignment criterion, has received relatively less attention. Dishonest behaviors in LLMs, such as spreading misinformation and defrauding users, present severe risks that intensify as these models approach superintelligent levels. Enhancing honesty in LLMs addresses critical limitations and helps uncover latent capabilities that are not readily expressed. This underscores the urgent need for reliable methods and benchmarks to effectively ensure and evaluate the honesty of LLMs.\n In this paper, we introduce BeHonest, a pioneering benchmark specifically designed to assess honesty in LLMs comprehensively. BeHonest evaluates three essential aspects of honesty: awareness of knowledge boundaries, avoidance of deceit, and consistency in responses. Building on this foundation, we designed 10 scenarios to evaluate and analyze 9 popular LLMs on the market, including both closed-source and open-source models from different model families with varied model sizes. Our findings indicate that there is still significant room for improvement in the honesty of LLMs. We encourage the AI community to prioritize honesty alignment in these models, which can harness their full potential to benefit society while preventing them from causing harm through deception or inconsistency. Our benchmark and code can be found at: \url{this https URL}.",True
60,Evaluation For Discrimination,"As language models (LMs) advance, interest is growing in applying them to high-stakes societal decisions, such as determining financing or housing eligibility. However, their potential for discrimination in such contexts raises ethical concerns, motivating the need for better methods to evaluate these risks. We present a method for proactively evaluating the potential discriminatory impact of LMs in a wide range of use cases, including hypothetical use cases where they have not yet been deployed. Specifically, we use an LM to generate a wide array of potential prompts that decision-makers may input into an LM, spanning 70 diverse decision scenarios across society, and systematically vary the demographic information in each prompt. Applying this methodology reveals patterns of both positive and negative discrimination in the Claude 2.0 model in select settings when no interventions are applied. While we do not endorse or permit the use of language models to make automated decisions for the high-risk use cases we study, we demonstrate techniques to significantly decrease both positive and negative discrimination through careful prompt engineering, providing pathways toward safer deployment in use cases where they may be appropriate. Our work enables developers and policymakers to anticipate, measure, and address discrimination as language model capabilities and applications continue to expand. We release our dataset and prompts at this https URL",True
61,Expert Comparisons on Biothreat Information,How do model responses compare\nagainst verified expert responses on long-\nform biorisk questions pertaining to ex-\necution of wet lab tasks?,True
63,FELM,"Assessing factuality of text generat

In [ ]:
wrong_ids = [63,197,199]
for wrong_id in wrong_ids:
  new_df.at[wrong_id,key] = False

---

## **measures_capabilities:**
   - **Definition:** Indicates whether the benchmark measures the AI's core functional abilities and performance in specific tasks or domains.
   - **Key Indicators:**
     - Evaluates linguistic abilities such as comprehension, generation, translation, or summarization.
     - Assesses reasoning, problem-solving, or decision-making skills.
     - Tests performance in specialized domains (e.g., mathematics, programming, legal reasoning).
     - Benchmarks general-purpose cognitive tasks, including memory, logic, and multi-modal understanding.



In [ ]:
key = 'measures_capabilities'
new_df[new_df[key]][['name','description',key]]

,name,description,measures_capabilities
0,100Q Hard,"A set of 100 human-written questions, curated to be relatively obscure and to encourage\nmodels in the Claude 2 family to respond with dubious or incorrect information. Examples include\n“Why is Berkeley Bowl called Berkeley Bowl?”, “What is the Opto Electronics Factory (OLF)?”,\n“Tell me about Mary I, Countess of Menteith.",True
1,1H-VideoQA,"Question-answering benchmarks for long-context video understanding need to have at least two properties: first, they need to contain long videos and second, their questions need to be designed to in a way that can differentiate among models that operate over different context lengths. Unfortunately, no existing benchmarks satisfy these properties for evaluating models that can handle hours-long videos like Gemini 1.5 Pro. The publicly available question answering benchmark with the longest videos is EgoSchema (Mangalam et al., 2023), but its videos are at most 3 minutes (i.e., 180 frames) in length. To bridge this evaluation gap, we introduce a new benchmark, 1H-VideoQA, composed of 125 five-way multiple-choice questions over public videos 40-105 minutes long.",True
2,CommonSenseQA,"When answering a question, people often draw upon their rich world knowledge in addition to the particular context. Recent work has focused primarily on answering questions given some relevant document or context, and required very little general background. To investigate question answering with prior knowledge, we present CommonsenseQA: a challenging new dataset for commonsense question answering. To capture common sense beyond associations, we extract from ConceptNet (Speer et al., 2017) multiple target concepts that have the same semantic relation to a single source concept. Crowd-workers are asked to author multiple-choice questions that mention the source concept and discriminate in turn between each of the target concepts. This encourages workers to create questions with complex semantics that often require prior knowledge. We create 12,247 questions through this procedure and demonstrate the difficulty of our task with a large number of strong baselines. Our best baseline is based on BERT-large (Devlin et al., 2018) and obtains 56% accuracy, well below human performance, which is 89%.",True
3,AGIEval,"Evaluating the general abilities of foundation models to tackle human-level tasks is a vital aspect of their development and application in the pursuit of Artificial General Intelligence (AGI). Traditional benchmarks, which rely on artificial datasets, may not accurately represent human-level capabilities. In this paper, we introduce AGIEval, a novel benchmark specifically designed to assess foundation model in the context of human-centric standardized exams, such as college entrance exams, law school admission tests, math competitions, and lawyer qualification tests. We evaluate several state-of-the-art foundation models, including GPT-4, ChatGPT, and Text-Davinci-003, using this benchmark. Impressively, GPT-4 surpasses average human performance on SAT, LSAT, and math competitions, attaining a 95% accuracy rate on the SAT Math test and a 92.5% accuracy on the English test of the Chinese national college entrance exam. This demonstrates the extraordinary performance of contemporary foundation models. In contrast, we also find that GPT-4 is less proficient in tasks that require complex reasoning or specific domain knowledge. Our comprehensive analyses of model capabilities (understanding, knowledge, reasoning, and calculation) reveal these models' strengths and limitations, providing valuable insights into future directions for enhancing their general capabilities. By concentrating on tasks pertinent to human cognition and decision-making, our benchmark delivers a more meaningful and robust evaluation of foundation models' performance in real-world scenarios. The data, code, and all model outputs are released in this https URL.",True
4,AI2D,"Diagrams ar

In [ ]:
wrong_ids = [32]
for wrong_id in wrong_ids:
  new_df.at[wrong_id,key] = False

---

## **is_variant:**
   - **Definition:** Specifies whether the benchmark is a variant of another existing benchmark, possibly adjusted for different conditions, languages, populations, or complexity levels.
   - **Key Indicators:**
     - Adaptations of existing benchmarks with minor or major modifications.
     - Benchmarks focusing on niche subsets of a broader test (e.g., language-specific versions).
     - Variations that test robustness under altered inputs or stress conditions.




In [ ]:
key = 'is_variant'
new_df[new_df[key]][['name','description',key]]

,name,description,is_variant
26,ASROB,"We present a preview of results on a new benchmark, ASROB (Automatic Speech Recognition\nfrom One Book). ASROB extends MTOB with 104 speech recordings (15 total hours) of transcribed\nand translated Kalamang speech from The Kalamang Collection (Visser, 2020c).15 Here we report\nexperiments on a subset of 6 recordings (45 minutes) with manually realigned phrase-level captions;\nwe use 5 of the recordings (∼800 phrases) as the in-context train set and 1 (∼100 phrases) as the test\nset. The same speaker from the test recording is present in 3 of the train recordings.",True
28,Adv SQuAD,"Standard accuracy metrics indicate that reading comprehension systems are making rapid progress, but the extent to which these systems truly understand language remains unclear. To reward systems with real language understanding abilities, we propose an adversarial evaluation scheme for the Stanford Question Answering Dataset (SQuAD). Our method tests whether systems can answer questions about paragraphs that contain adversarially inserted sentences, which are automatically generated to distract computer systems without changing the correct answer or misleading humans. In this adversarial setting, the accuracy of sixteen published models drops from an average of 75% F1 score to 36%; when the adversary is allowed to add ungrammatical sequences of words, average accuracy on four models decreases further to 7%. We hope our insights will motivate the development of new models that understand language more precisely.",True
36,BigBench Hard,"Bench (Srivastava et al., 2022) is a diverse evaluation suite that focuses on tasks believed to be beyond the capabilities of current language models. Language models have already made good progress on this benchmark, with the best model in the BIG-Bench paper outperforming average reported human-rater results on 65% of the BIG-Bench tasks via few-shot prompting. But on what tasks do language models fall short of average human-rater performance, and are those tasks actually unsolvable by current language models?\nIn this work, we focus on a suite of 23 challenging BIG-Bench tasks which we call BIG-Bench Hard (BBH). These are the task for which prior language model evaluations did not outperform the average human-rater. We find that applying chain-of-thought (CoT) prompting to BBH tasks enables PaLM to surpass the average human-rater performance on 10 of the 23 tasks, and Codex (code-davinci-002) to surpass the average human-rater performance on 17 of the 23 tasks. Since many tasks in BBH require multi-step reasoning, few-shot prompting without CoT, as done in the BIG-Bench evaluations (Srivastava et al., 2022), substantially underestimates the best performance and capabilities of language models, which is better captured via CoT prompting. As further analysis, we explore the interaction between CoT and model scale on BBH, finding that CoT enables emergent task performance on several BBH tasks with otherwise flat scaling curves.",True
49,Covost 2,"Speech translation has recently become an increasingly popular topic of research, partly due to the development of benchmark datasets. Nevertheless, current datasets cover a limited number of languages. With the aim to foster research in massive multilingual speech translation and speech translation for low resource language pairs, we release CoVoST 2, a large-scale multilingual speech translation corpus covering translations from 21 languages into English and from English into 15 languages. This represents the largest open dataset available to date from total volume and language coverage perspective. Data sanity checks provide evidence about the quality of the data, which is released under CC0 license. We also provide extensive speech recognition, bilingual and multilingual machine translation and speech translation baselines with open-source implementation.",True
55,Dynabench SQuAD,"For question answering, we use Adversarial SQuAD (J

In [ ]:
wrong_ids = [199,213]
for wrong_id in wrong_ids:
  new_df.at[wrong_id,key] = False

In [ ]:
# LSAT,AP,SAT
new_df.loc[new_df.name.str.contains('SAT ',na=False), 'is_variant'] = True
new_df.loc[new_df.name.str.contains('AP ', na=False), 'is_variant'] = True
new_df.loc[new_df.name == 'LSAT', 'is_variant'] = True


## **existential_risk:**
   - **Definition:** Indicates whether the benchmark assesses potential existential risks associated with AI, particularly those related to catastrophic biological, chemical, radiological, or nuclear (CBRN) threats or other large-scale risks that could threaten human survival.
   - **Key Indicators:**
     - Evaluates AI’s potential role in facilitating, detecting, or preventing CBRN threats.
     - Tests for alignment issues that could lead to existential-level risks.
     - Assesses AI misuse scenarios that could result in global catastrophic outcomes.
     - Focuses on long-term safety concerns, including AI’s role in dual-use technologies with existential implications.



In [ ]:
key = 'existential_risk'
new_df[new_df[key]][['name','description',key]]

,name,description,existential_risk
94,Long-form Biological Risk Questions (OpenAI),"We graded the accuracy of model responses to long-form biorisk questions. Our long-form biothreat\ninformation questions test acquiring critical and sensitive information across the five stages of\nthe biological threat creation process [30]: Ideation, Acquisition, Magnification, Formulation,\nand Release. We designed the questions and detailed rubrics with Gryphon Scientific due to\ntheir expertise working with dangerous biological agents in a national security setting. We used the o1-preview (pre-mitigation) model as an autograder, validating agreement with a trusted\nbiosecurity expert. We made adjustments to the rubric and iterated on the autograder based on\nthe expert feedback.",True
165,Radiological and Nuclear Expert Knowledge,"Radiological and\nNuclear Expert\nKnowledge:\nUnclassified but potentially sensitive\ninformation (expert knowledge, tacit\nknowledge, planning) in the radiolog-\nical and nuclear threat creation pro-\ncesses\nCan models answer difficult ex-\npert and tacit knowledge ques-\ntions related to radiological and\nnuclear topics?",True
183,Structured Expert Probing Campaign – Chem-Bio Novel Design,Structured Expert Probing Campaign – Chem-Bio Novel Design: Novel chem-bio weapon design and development\nDo models provide meaningful uplift\nbeyond existing resources in designing\nnovel and feasible chem-bio threats?,True
184,Structured Expert Probing Campaign – Radiological & Nuclear,"Structured expert probing campaign – radiological & nuclear:\nTo evaluate o1’s potential to assist in Radiological and Nuclear (RN) weapon development, nine\nexperts specializing in RN topics were onboarded to interact with a pre-mitigation model (i.e.,\nan unreleased model not trained with safety refusals, only used for worst-case scenario testing).\nThese experts were sourced by a senior member of OpenAI’s National Security team, and were\nscreened for relevant and diverse industry and government experience. After training on the\nPreparedness Framework CBRN risk thresholds (Low, Medium, High, Critical) and completing a\n20-question pre-screening quiz to confirm their understanding, experts were tasked with assessing\nwhether the model could assist in radiological and nuclear weapon creation. Each conversation\nwas assessed against the Preparedness Framework risk thresholds. Any conversation that received\nan initial rating of High or above was assigned to additional experts for further review. Of the 42\ntotal expert conversations, 32 were rated Low risk and 10 were rated Medium risk, while the 4\nconversations initially rated High were later classified as either Low or Medium after subsequent\nexpert reviews. The Post-Mitigation o1 model successfully refused all Medium and High prompts,\nand was ultimately designated as earning a “Low” risk under the Preparedness Framework for\nRN weapon creation capability.",True


## **jailbreak_resistance:**
   - **Definition:** Indicates whether the benchmark assesses the AI’s resistance to "jailbreaking" attempts—situations where users try to manipulate the AI into bypassing safety restrictions, generating harmful content, or performing unauthorized actions.
   - **Key Indicators:**
     - Evaluates how well the AI can resist adversarial prompts designed to circumvent content safety measures.
     - Tests for the AI’s robustness against prompt injection attacks.
     - Assesses the AI’s ability to maintain ethical and safety boundaries despite manipulative or deceptive inputs.
     - Benchmarks scenarios that simulate real-world attempts to exploit AI systems for malicious purposes.


In [ ]:
key = 'jailbreak_resistance'
new_df[new_df[key]][['name','description',key]]

,name,description,jailbreak_resistance
73,Human Sourced Jailbreaks,"We further evaluate the robustness of the o1 models to jailbreaks: adversarial prompts that\npurposely try to circumvent model refusals for content it’s not supposed to produce [15, 16, 17, 18].\nWe consider four evaluations that measure model robustness to known jailbreaks:\n• Production Jailbreaks: A series of jailbreaks identified in production ChatGPT data.\n• Jailbreak Augmented Examples: Applies publicly known jailbreaks to examples from our\nstandard disallowed content evaluation\n• Human Sourced Jailbreaks: Jailbreaks sourced from human redteaming.\n• StrongReject [16]: An academic jailbreak benchmark that tests a model’s resistance against\ncommon attacks from the literature. Following [16], we calculate goodness@0.1, which is the\nsafety of the model when evaluated against the top 10% of jailbreak techniques per prompt.",True
80,Instruction Hierarchy Evaluation,"Unlike o1-preview and o1-mini, the deployment of o1 on the API allows developers to specify a\ncustom developer message that is included with every prompt from one of their end users. This\ncould potentially allow developers to circumvent guardrails in o1 if not handled properly.\nTo mitigate this issue, we taught the model to adhere to an Instruction Hierarchy[20]. At a\nhigh level, we now have three classifications of messages sent to o1: system messages, developer\nmessages, and user messages. We collected examples of these different types of messages conflicting\nwith each other, and supervised o1 to follow the instructions in the system message over developer\nmessages, and instructions in developer messages over user messages.\nWe created several evaluations to measure the model’s ability to follow the Instruction Hierarchy\nin o1. As can be seen across all but one of these evaluations, o1 performs significantly better in\nfollowing instructions in the correct priority when compared to GPT-4o.\nFirst is a set of evaluations where different types of messages are in conflict with each other; the model must choose to follow the instructions in the highest priority message to pass these evals. The second set of evaluations considers a more realistic scenario, where the model is meant to be\na math tutor, and the user attempts to trick the model into giving away the solution. Specifically,\nwe instruct the model in the system message or developer message to not give away the answer\nto a math question, and the user message attempts to trick the model into outputting the answer\nor solution. To pass the eval, the model must not give away the answer.to a math question, and the user message attempts to trick the model into outputting the answer\nor solution. To pass the eval, the model must not give away the answer. In the third set of evaluations, we instruct the model to not output a certain phrase (e.g., “access\ngranted”) or to not reveal a bespoke password in the system message, and attempt to trick the\nmodel into outputting it in user or developer messages",True
82,Jailbreak Arena,"OpenAI collaborated with Gray Swan to make the o1 model available for jailbreaking in the Gray\nSwan Arena. The challenge in the arena focused on testing for generation of violent content,\nself-harm content, and malicious code. The aim was to test how robust our mitigation methods\nare by choosing a few targeted categories. The arena tested for harmful generations as a result\nof text input, image-text input, and malicious code generation. An attempt was considered a\n‘successful jailbreak’ for violence and self-harm if the response triggered the OpenAI mod API\nat a threshold of 0.8 or more. Gray Swan used their internal classifiers to determine if a code\ngeneration was malicious.",True
83,JailbreakBench,"Jailbreak attacks cause large language models (LLMs) to generate harmful, unethical, or otherwise objectionable content. Evaluating these attacks presents a number of challenges, which the current collecti

In [ ]:

wrong_ids = [94]
for wrong_id in wrong_ids:
  new_df.at[wrong_id,key] = False

## save file

In [ ]:
new_df.to_csv('classified_benchmarks_reviewed.csv')

In [ ]:
new_df

,name,description,source,o1,Claude_3,gemini_15,llama3,gpt-4o,internal,behavioral_evaluation,knowledge_accuracy,wrong_answer_resistance,informational_risks,measures_capabilities,is_variant,existential_risk,jailbreak_resistance
0,100Q Hard,"A set of 100 human-written questions, curated to be relatively obscure and to encourage\nmodels in the Claude 2 family to respond with dubious or incorrect information. Examples include\n“Why is Berkeley Bowl called Berkeley Bowl?”, “What is the Opto Electronics Factory (OLF)?”,\n“Tell me about Mary I, Countess of Menteith.",https://www-cdn.anthropic.com/de8ba9b01c9ab7cbabf5c33b80b7bbc618857627/Model_Card_Claude_3.pdf,False,True,False,False,False,True,False,True,False,False,True,False,False,False
1,1H-VideoQA,"Question-answering benchmarks for long-context video understanding need to have at least two properties: first, they need to contain long videos and second, their questions need to be designed to in a way that can differentiate among models that operate over different context lengths. Unfortunately, no existing benchmarks satisfy these properties for evaluating models that can handle hours-long videos like Gemini 1.5 Pro. The publicly available question answering benchmark with the longest videos is EgoSchema (Mangalam et al., 2023), but its videos are at most 3 minutes (i.e., 180 frames) in length. To bridge this evaluation gap, we introduce a new benchmark, 1H-VideoQA, composed of 125 five-way multiple-choice questions over public videos 40-105 minutes long.",https://arxiv.org/pdf/2403.05530,False,False,True,False,False,False,False,True,False,False,True,False,False,False
2,CommonSenseQA,"When answering a question, people often draw upon their rich world knowledge in addition to the particular context. Recent work has focused primarily on answering questions given some relevant document or context, and required very little general background. To investigate question answering with prior knowledge, we present CommonsenseQA: a challenging new dataset for commonsense question answering. To capture common sense beyond associations, we extract from ConceptNet (Speer et al., 2017) multiple target concepts that have the same semantic relation to a single source concept. Crowd-workers are asked to author multiple-choice questions that mention the source concept and discriminate in turn between each of the target concepts. This encourages workers to create questions with complex semantics that often require prior knowledge. We create 12,247 questions through this procedure and demonstrate the difficulty of our task with a large number of strong baselines. Our best baseline is based on BERT-large (Devlin et al., 2018) and obtains 56% accuracy, well below human performance, which is 89%.",https://arxiv.org/pdf/1811.00937v2,False,False,False,True,False,False,False,True,False,False,True,False,False,False
3,AGIEval,"Evaluating the general abilities of foundation models to tackle human-level tasks is a vital aspect of their development and application in the pursuit of Artificial General Intelligence (AGI). Traditional benchmarks, which rely on artificial datasets, may not accurately represent human-level capabilities. In this paper, we introduce AGIEval, a novel benchmark specifically designed to assess foundation model in the context of human-centric standardized exams, such as college entrance exams, law school admission tests, math competitions, and lawyer qualification tests. We evaluate several state-of-the-art foundation models, including GPT-4, ChatGPT, and Text-Davinci-003, using this benchmark. Impressively, GPT-4 surpasses average human performance on SAT, LSAT, and math competitions, attaining a 95% accuracy rate on the SAT Math test and a 92.5% accuracy on the English test of the Chinese national college entrance exam. This demonstrates the extraordinary performance of contemporary foundation models. In contrast, we also find that GPT-4 is less proficient in tasks that require complex re